In [3]:
# ods -> dwd 层： 执行数据清洗任务
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ods_to_dwd") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("set hive.exec.dynamici.partition=true;")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict;")

print("DWD init spark content successful")

DWD init spark content successful


In [13]:
# create database time_series
spark.sql("DROP TABLE IF EXISTS dwd_time_series;")
sql = """
CREATE TABLE IF NOT EXISTS dwd_time_series
(id bigint,
time TIMESTAMP,
lon float,
lat float)
PARTITIONED BY(yearMonth string)
STORED AS PARQUET;
"""
spark.sql(sql)

# create database job
spark.sql("DROP TABLE IF EXISTS dwd_job;")
sql = """
CREATE TABLE IF NOT EXISTS dwd_job
(id bigint,start_time TIMESTAMP,end_time TIMESTAMP)
PARTITIONED BY(yearMonth string)
STORED AS PARQUET;
"""
spark.sql(sql)

# 创建输出表
#CREATE TABLE output(id SERIAL PRIMARY KEY,job_id int4);

print("Init dwd dwd_time_series ,dwd_job  Success")

Init dwd dwd_time_series ,dwd_job  Success


In [15]:
# ODS 表写入DWD 表 dwd_job


sql = '''
INSERT INTO dwd_job partition(yearMonth)
SELECT
 id as id,
 start_time as start_time,
 end_time as end_time,
 date_format(start_time,'yyyyMM') as yearMonth
 FROM ods_job;
'''
print(sql)
result = spark.sql(sql)

# 查看一条数据
result = spark.sql("SELECT * FROM dwd_job LIMIT 1")
result.show()


INSERT INTO dwd_job partition(yearMonth)
SELECT
 id as id,
 start_time as start_time,
 end_time as end_time,
 date_format(start_time,'yyyyMM') as yearMonth
 FROM ods_job;

+-----+-------------------+-------------------+---------+
|   id|         start_time|           end_time|yearMonth|
+-----+-------------------+-------------------+---------+
|30656|2027-07-01 00:00:00|2027-07-01 01:00:00|   202707|
+-----+-------------------+-------------------+---------+



In [16]:
# ODS 表写入DWD 表 dwd_time_series


sql = '''
INSERT INTO dwd_time_series partition(yearMonth)
SELECT
 id as id,
 time as time,
 lon as lon,
 lat as lat,
 date_format(time,'yyyyMM') as yearMonth
 FROM ods_time_series;
'''
print(sql)
result = spark.sql(sql)

# 查看一条数据
result = spark.sql("SELECT * FROM dwd_time_series LIMIT 1")
result.show()


INSERT INTO dwd_time_series partition(yearMonth)
SELECT
 id as id,
 time as time,
 lon as lon,
 lat as lat,
 date_format(time,'yyyyMM') as yearMonth
 FROM ods_time_series;

+-------+-------------------+---------+---------+---------+
|     id|               time|      lon|      lat|yearMonth|
+-------+-------------------+---------+---------+---------+
|7349759|2025-03-01 00:00:00|45.117603|46.117603|   202503|
+-------+-------------------+---------+---------+---------+



In [19]:
# 查看分区数目
result = spark.sql("SHOW PARTITIONS dwd_time_series;")
result.show()

+----------------+
|       partition|
+----------------+
|yearMonth=202312|
|yearMonth=202401|
|yearMonth=202402|
|yearMonth=202403|
|yearMonth=202404|
|yearMonth=202405|
|yearMonth=202406|
|yearMonth=202407|
|yearMonth=202408|
|yearMonth=202409|
|yearMonth=202410|
|yearMonth=202411|
|yearMonth=202412|
|yearMonth=202501|
|yearMonth=202502|
|yearMonth=202503|
|yearMonth=202504|
|yearMonth=202505|
|yearMonth=202506|
|yearMonth=202507|
+----------------+
only showing top 20 rows



In [20]:
# 查看分区数目
result = spark.sql("select date_format(time,'yyyyMM') from dwd_time_series ;")
result.show()

+-------------------------+
|date_format(time, yyyyMM)|
+-------------------------+
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
|                   202503|
+-------------------------+
only showing top 20 rows

